In [1]:
import pandas as pd 
from datetime import datetime


df= pd.read_csv('data/dataELO.csv')
df['Data'] = pd.to_datetime(df['Data'], infer_datetime_format=True, errors='coerce')
df.tail(20)

C:\Users\Cristina Jorge\AppData\Local\Temp\ipykernel_1484\3007485351.py:6: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['Data'] = pd.to_datetime(df['Data'], infer_datetime_format=True, errors='coerce')


,Comp,Temp,Data,Casa,GC,GF,Fora,Res,PC,PF,J
24006,Série B,2023.0,2023-11-10,Vila Nova-GO,4.0,1.0,Londrina,VC,3.0,0.0,351.0
24007,Série B,2023.0,2023-11-10,Sport,0.0,0.0,Atlético-GO,E,1.0,1.0,352.0
24008,Série A,2023.0,2023-11-11,Flamengo,1.0,1.0,Fluminense,E,1.0,1.0,326.0
24009,Série A,2023.0,2023-11-11,Coritiba,1.0,0.0,Cruzeiro,VC,3.0,0.0,328.0
24010,Série B,2023.0,2023-11-11,Tombense,0.0,1.0,Ponte Preta,VV,0.0,3.0,356.0
24011,Série B,2023.0,2023-11-11,Chapecoense,2.0,0.0,Botafogo-SP,VC,3.0,0.0,355.0
24012,Série A,2023.0,2023-11-11,Palmeiras,3.0,0.0,Internacional,VC,3.0,0.0,327.0
24013,Série B,2023.0,2023-11-11,Ituano,2.0,1.0,Sampaio Corrêa,VC,3.0,0.0,354.0
24014,Série B,2023.0,2023-11-11,Avaí,1.0,1.0,CRB,E,1.0,1.0,353.0
24015,Série A,2023.0,2023-11-12,Bahia,1.0,1.0,Athletico-PR,E,1.0,1.0,331.0


In [23]:
df['GC'] = df['GC'].fillna(0).astype(int)
df['GF'] = df['GF'].fillna(0).astype(int)
df.dtypes

Comp                     object
Temp                    float64
Data             datetime64[ns]
Casa                     object
GC                        int32
GF                        int32
Fora                     object
Res                      object
PC                      float64
PF                      float64
J                       float64
Elo Casa                float64
Elo Fora                float64
Novo Elo Casa           float64
Novo Elo Fora           float64
dtype: object

In [24]:
#This function updates the Elo ratings and creates a ranking

timestamp_df = pd.DataFrame(columns=['Timestamp'])
def update_elo(df):
    #Gives different weights to each competition
    k_factor_map = {
                    'Série C': 10, 
                    'Série B': 20, 
                    'CBR': 25, 
                    'Série A': 30, 
                    }
    teams = {}
    elo_ranking = {}
    average_elo = {}
    team_games = {}
    team_victories = {}
    team_losses = {}
    team_goals = {}
    team_draws = {}
    last_competition = {}
    current_year = datetime.now().year
    
    #Creates a list of teams and gives then an initial rating of 1500
    for index, row in df.iterrows():
        home_team, away_team = row['Casa'], row['Fora']
        comp_level = row['Comp']
        match_date = row['Data'].year
        k_factor = k_factor_map.get(comp_level, 50) 
        if home_team not in teams:
            teams[home_team] = 1500
            elo_ranking[home_team] = teams[home_team]
        if away_team not in teams:
            teams[away_team] = 1500
            elo_ranking[away_team] = teams[away_team]
        home_elo, away_elo = teams[home_team], teams[away_team]
        home_score, away_score = row['GC'], row['GF']
        result = row['Res']
        # Only update the last competition if it's not 'CBR' and the match is in the current year
        if comp_level != 'CBR' and match_date == current_year:
            last_competition[home_team] = comp_level
            last_competition[away_team] = comp_level

        #Gives different weights for home and away games
        if result == 'VC':
            home_result, away_result = 1, 0
        elif result == 'VV':
            home_result, away_result = 0, 1
        else:
            home_result, away_result = 0.5, 0.5
        home_expected = 1 / (1 + 10 ** ((away_elo - home_elo - 100) / 400))
        away_expected = 1 / (1 + 10 ** ((home_elo - away_elo + 100) / 400))
        
        # Calculates the number of goals
        
        if home_team in team_goals:
            team_goals[home_team] += home_score
        else:
            team_goals[home_team] = home_score

        if away_team in team_goals:
            team_goals[away_team] += away_score
        else:
            team_goals[away_team] = away_score

            team_goals[away_team] = away_score
       # Update the number of victories, losses, and draws for each team
        if result == 'VC':
            home_result, away_result = 1, 0
            if home_team in team_victories:
                team_victories[home_team] += 1
            else:
                team_victories[home_team] = 1
            if away_team in team_losses:
                team_losses[away_team] += 1
            else:
                team_losses[away_team] = 1
        elif result == 'VV':
            home_result, away_result = 0, 1
            if home_team in team_losses:
                team_losses[home_team] += 1
            else:
                team_losses[home_team] = 1
            if away_team in team_victories:
                team_victories[away_team] += 1
            else:
                team_victories[away_team] = 1
        else:
            home_result, away_result = 0.5, 0.5
            if home_team in team_draws:
                team_draws[home_team] += 1
            else:
                team_draws[home_team] = 1
            if away_team in team_draws:
                team_draws[away_team] += 1
            else:
                team_draws[away_team] = 1
        # Update the number of games played for each team
        if home_team in team_games:
            team_games[home_team] += 1
        else:
            team_games[home_team] = 1
        if away_team in team_games:
            team_games[away_team] += 1
        else:
            team_games[away_team] = 1
        
        # Calculate the goal difference and adjust the K factor accordingly
        goal_diff = abs(home_score - away_score)
        if goal_diff >= 4:
            k_factor = k_factor + 0.75 + (goal_diff - 4) / 8
        elif goal_diff == 3:
            k_factor = k_factor + 0.75
        elif goal_diff == 2:
            k_factor = k_factor + 0.5
        else:
            k_factor = k_factor
            
        # Calculate the rating difference factor
        rating_diff = abs(home_elo - away_elo)
        if rating_diff > 400:
            rating_diff = 400
        rating_factor = rating_diff / 400 + 1
            
        teams[home_team] = home_elo + rating_factor * k_factor * (home_result - home_expected)
        teams[away_team] = away_elo + rating_factor * k_factor * (away_result - away_expected)
        teams[home_team] = round(teams[home_team], 2)
        teams[away_team] = round(teams[away_team], 2)
        elo_ranking[home_team] = teams[home_team]
        elo_ranking[away_team] = teams[away_team]
        df.at[index, 'Elo Casa'] = home_elo
        df.at[index, 'Elo Fora'] = away_elo
        df.at[index, 'Novo Elo Casa'] = teams[home_team]
        df.at[index, 'Novo Elo Fora'] = teams[away_team]

        
        for team, elo in teams.items():
            if team in average_elo:
                average_elo[team].append(elo)
            else:
                average_elo[team] = [elo]

    # Compute average Elo for each team
    for team, elo_list in average_elo.items():
        average_elo[team] = round(sum(elo_list) / len(elo_list), 2)
        
  

    #Createa the Datasets
    elo_df = pd.DataFrame.from_dict(elo_ranking, orient='index', columns=['Elo'])
    elo_df = elo_df.sort_values(by='Elo', ascending=False)
    elo_df.index.name = 'Time'
    elo_df.reset_index(inplace=True)
    #elo_df = elo_df[~elo_df['Time'].str.contains(r'\(ARG\)|\(BOL\)|\(CHI\)|\(COL\)|\(EQU\)|\(MEX\)|\(PAR\)|\(PER\)|\(URU\)|\(VEN\)', regex=True)]
    elo_df['Jogos'] = elo_df['Time'].map(team_games) 
    elo_df = elo_df[elo_df['Jogos'] >= 30]
    elo_df['Rank'] = elo_df['Elo'].rank(ascending=False, method='dense')
    elo_df['Média'] = elo_df['Time'].map(average_elo).round(2)
    elo_df['#'] = elo_df['Média'].rank(ascending=False, method='dense')
    elo_df['Gols'] = elo_df['Time'].map(team_goals)
    elo_df['Vitórias'] = elo_df['Time'].map(team_victories)
    elo_df['Derrotas'] = elo_df['Time'].map(team_losses)
    elo_df['Empates'] = elo_df['Time'].map(team_draws)
    elo_df['Série'] = elo_df.apply(lambda row: last_competition.get(row['Time'], '-'), axis=1) 
    elo_df.replace({'Série A': 'A','Série B': 'B','Série C': 'C',                   
            }, inplace=True)
    timestamp_now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    timestamp_df.loc[len(timestamp_df)] = [timestamp_now]

    
    return df, elo_df

df, elo_df = update_elo(df)
df.to_csv('data/dataELOranking.csv', index=False)
elo_df.to_json('data/dataELOranking.json', orient='records')
timestamp_df.to_json('data/timestamps.json')



In [25]:
filt = (elo_df['Série'] == "A")
elo_df[filt]

,Time,Elo,Jogos,Rank,Média,#,Gols,Vitórias,Derrotas,Empates,Série
0,Palmeiras,1868.74,954,1.0,1722.57,2.0,1399,436,247,271,A
1,Flamengo,1846.87,991,2.0,1721.56,3.0,1374,428,263,300,A
2,Red Bull Bragantino,1837.47,641,3.0,1572.20,27.0,829,248,228,165,A
3,Atlético-MG,1834.38,977,4.0,1701.50,7.0,1413,408,291,278,A
4,Grêmio,1807.55,956,5.0,1706.23,5.0,1264,405,273,278,A
5,São Paulo,1806.15,940,6.0,1727.47,1.0,1296,406,244,290,A
6,Corinthians,1804.85,968,7.0,1711.22,4.0,1225,398,259,311,A
7,Botafogo,1786.41,961,8.0,1660.42,12.0,1242,356,311,294,A
8,Athletico-PR,1782.16,879,9.0,1668.62,11.0,1163,350,309,220,A
9,Fluminense,1769.23,983,10.0,1686.21,9.0,1310,383,313,287,A


In [10]:
df

,Comp,Temp,Data,Casa,GC,GF,Fora,Res,PC,PF,J,Elo Casa,Elo Fora,Novo Elo Casa,Novo Elo Fora
0,CBR,2003.0,2003-02-05,CFZ-DF,0.0,0.0,Fortaleza,E,1.0,1.0,11.0,1500.00,1500.00,1496.50,1503.50
1,CBR,2003.0,2003-02-05,Itabaiana,0.0,1.0,Vasco,VV,0.0,3.0,8.0,1500.00,1500.00,1484.00,1516.00
2,CBR,2003.0,2003-02-05,CENE,0.0,2.0,Botafogo,VV,0.0,3.0,13.0,1500.00,1500.00,1483.68,1516.32
3,CBR,2003.0,2003-02-05,Ypiranga-AP,1.0,3.0,Remo,VV,0.0,3.0,9.0,1500.00,1500.00,1483.68,1516.32
4,CBR,2003.0,2003-02-05,Operário-MT,0.0,1.0,Palmeiras,VV,0.0,3.0,6.0,1500.00,1500.00,1484.00,1516.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24021,Série B,2023.0,2023-11-12,Novorizontino,1.0,2.0,Vitória,VV,0.0,3.0,357.0,1590.85,1645.54,1578.01,1658.38
24022,Série B,2023.0,2023-11-12,Ceará,2.0,0.0,Mirassol,VC,3.0,0.0,358.0,1612.14,1617.11,1619.75,1609.50
24023,Série A,2023.0,2023-11-12,Grêmio,0.0,1.0,Corinthians,VV,0.0,3.0,329.0,1831.31,1781.09,1807.55,1804.85
24024,Série B,2023.0,2023-11-14,Guarani,1.0,1.0,Criciúma,E,1.0,1.0,359.0,1634.52,1640.75,1631.84,1643.43


In [5]:
elo_df

,Time,Elo,Jogos,Rank,Média,#,Gols,Vitórias,Derrotas,Empates,Série
0,Palmeiras,1868.74,954,1.0,1722.57,2.0,NaN,436,247,271,A
1,Flamengo,1846.87,991,2.0,1721.56,3.0,NaN,428,263,300,A
2,Red Bull Bragantino,1837.47,641,3.0,1572.20,27.0,829.0,248,228,165,A
3,Atlético-MG,1834.38,977,4.0,1701.50,7.0,NaN,408,291,278,A
4,Grêmio,1807.55,956,5.0,1706.23,5.0,NaN,405,273,278,A
...,...,...,...,...,...,...,...,...,...,...,...
412,Campinense,1402.43,150,132.0,1458.65,114.0,171.0,46,68,36,-
413,Fast Clube,1400.07,47,133.0,1429.74,131.0,56.0,14,23,10,-
415,Mogi Mirim,1389.42,173,134.0,1425.27,132.0,184.0,47,76,50,-
417,Guaratinguetá,1379.66,240,135.0,1444.27,127.0,290.0,80,105,55,-
